**Задание 1.**  

Используем файл keywords.csv.    
Необходимо написать гео-классификатор, который каждой строке сможет выставить географическую принадлежность определенному региону. Т. е. если поисковый запрос содержит название города региона, то в столбце 'region' пишется название этого региона. Если поисковый запрос не содержит названия города, то ставим 'undefined'. 

Правила распределения по регионам Центр, Северо-Запад и Дальний Восток: 

geo_data = {  
'Центр': ['москва', 'тула', 'ярославль'],  
'Северо-Запад': ['петербург', 'псков', 'мурманск'],  
'Дальний Восток': ['владивосток', 'сахалин', 'хабаровск']  
}

Результат классификации запишите в отдельный столбец region.

In [83]:
import pandas as pd
data = pd.read_csv('keywords.csv')

geo_data = {
    'Центр'         : ['москва', 'тула', 'ярославль'],
    'Северо-Запад'  : ['петербург', 'псков', 'мурманск'],
    'Дальний Восток': ['владивосток', 'сахалин', 'хабаровск']}

def region_name(row):

    for region, ValueList in geo_data.items():
        for i in ValueList:
            try%
            if i in row['keyword']:
                return region
            
    return 'undefined'

data['region'] = data.apply(region_name, axis=1)

data[369:371]

,keyword,shows,region
369,порно инцест,504714,undefined
370,авито ру санкт петербург,425134,Северо-Запад


**Задание 2**

Напишите функцию, которая классифицирует фильмы из материалов занятия по следующим правилам:**  
  - оценка 2 и меньше - низкий рейтинг  
  - оценка 4 и меньше - средний рейтинг  
  - оценка 4.5 и 5 - высокий рейтинг  

Результат классификации запишите в столбец class

In [5]:
import pandas as pd
from datetime import datetime, date, time
data = pd.read_csv('ratings.csv')

def class_name(row):

    if row['rating'] <= 2:
        return 'низкий рейтинг'
    if row['rating'] <= 4:
        return 'средний рейтинг'
    if row['rating'] > 4:
        return 'высокий рейтинг'
    
data['class'] = data.apply(class_name, axis=1)

data.head()

,userId,movieId,rating,timestamp,class
0,1,31,2.5,1260759144,средний рейтинг
1,1,1029,3.0,1260759179,средний рейтинг
2,1,1061,3.0,1260759182,средний рейтинг
3,1,1129,2.0,1260759185,низкий рейтинг
4,1,1172,4.0,1260759205,средний рейтинг


**Задание 3.  **

Посчитайте среднее значение Lifetime киноманов (пользователи, которые поставили 100 и более рейтингов). Под Lifetime понимается разница между максимальным и минимальным значением timestamp для каждого пользователя. Ответ дайте в днях. 

In [30]:
import pandas as pd
data = pd.read_csv('ratings.csv')

data_ = data.groupby('userId').agg({'timestamp': ['min', 'max']}).copy()

data_['Lifetime'] = data_.apply(
    lambda x: (datetime.fromtimestamp(x['timestamp']['max']) - 
               datetime.fromtimestamp(x['timestamp']['min'])).days, axis=1)
data_.head()

timestamp             Lifetime
               min         max         
userId                                 
1       1260759108  1260759205        0
2        835355395   835356246        0
3       1298861589  1298932787        0
4        949778714   949982274        2
5       1163373044  1163375145        0

**Задание 4**

Есть мнение, что "раньше снимали настоящее кино, не то что сейчас". Ваша задача проверить это утверждение, используя файлы с рейтингами фильмов из материалов занятия. Т. е. проверить верно ли, что с ростом года выпуска фильма его средний рейтинг становится ниже.

При этом мы не будем затрагивать субьективные факторы выставления этих рейтингов, а пройдемся по следующему алгоритму:

1. В переменную years запишите список из всех годов с 1950 по 2010.

2. Напишите функцию production_year, которая каждой строке из названия фильма выставляет год выпуска. Не все названия фильмов содержат год выпуска в одинаковом формате, поэтому используйте следующий алгоритм:
    - для каждой строки пройдите по всем годам списка years
    - если номер года присутствует в названии фильма, то функция возвращает этот год как год выпуска
    - если ни один из номеров года списка years не встретился в названии фильма, то возвращается 1900 год

3. Запишите год выпуска фильма по алгоритму пункта 2 в новый столбец 'year'

4. Посчитайте средний рейтинг всех фильмов для каждого значения столбца 'year' и отсортируйте результат по убыванию рейтинга

In [1]:
import pandas as pd
movies = pd.read_csv('movies.csv')
ratings = pd.read_csv('ratings.csv')

joined = ratings.merge(movies, on='movieId', how='left')

years = [str(x) for x in range(1950,2011)]

def production_year(row):
    for year in years:
        if year in row['title']:
            return year
        
    return 1990

joined['year'] = joined.apply(production_year, axis=1)

joined.groupby('year').agg({'rating': 'mean'}).sort_values('rating', ascending=True).head()

,rating
year,
2000,3.356104
1997,3.415764
1996,3.426600
2003,3.444777
2005,3.448434
